# Sequence Tagging With A LSTM-CRF

This is the fourth post in my series about named entity recognition. If you haven’t seen the last three, have a look now. The last time we used a recurrent neural network to model the sequence structure of our sentences. Now we use a hybrid approach combining a bidirectional LSTM model and a CRF model. The so called LSTM-CRF is a state-of-the-art approach to named entity recognition.

Let’s recall the situation from my post about conditional random fields. We are given a input sequence $x = (x_1,\dots, x_m)$, i.e. the words of a sentence and a sequence of output states $s = (s_1,\dots, s_m)$, i.e. the named entity tags. In conditional random fields we modeled the conditional probability


<center>\[p(s_1,\dots,s_m|x_1,\dots,x_m)\]</center>


of the output state sequence give a input sequence. We did this by defining a feature map


<center>\[\Phi(x_1,\dots,x_m,s_1,\dots,s_m)\in\mathbb{R}^d\]</center>  


that maps an entire input sequence x paired with an entire state sequence s to some d-dimensional feature vector. Then we can model the probability as a log-linear model with the parameter vector $w\in\mathbb{R}^d$


<center>\[p(s|x; w) = \frac{\exp(w\cdot\Phi(x, s))}{\sum_{s^\prime} \exp(w\cdot\Phi(x, s^\prime))},\]</center>


where $s^\prime$ ranges over all possible output sequences. We can view the expression $w\cdot\Phi(x, s) = \text{score}_{crf}(x,s)$ as a scoring how well the state sequence fits the given input sequence. The idea is now, to replace the linear scoring function by a non-linear neural network. So we define the score


<center>\[\text{score}_{lstm-crf}(x,s) = \sum_{i=0}^n W_{s_{i-1}, s_i} \cdot \text{LSTM}(x)_i + b_{s_{i-1},s_i},\]</center>


where $W_{s_{i-1}, s_i}$ and b are the weight vector and the bias corresponding to the transition from $s_{i-1}$ to $s_i$, respectively.


Note, that the score functions are also called $\textit{potential functions}$. After constructing this score function, we can optimize the conditional probability $p(s|x; W, b)$ like in the usual CRF and propagating back trough the network. We are going to use the implementation provided by the keras-contrib package, that contains useful extensions to the official keras package.

Now dive into the applied part. We start as always by loading the data.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("data/ner_dataset.csv", encoding="latin1")

In [3]:
data = data.fillna(method="ffill")

In [4]:
data.tail()

,Sentence #,Word,POS,Tag
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [5]:
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words); n_words

35179

In [6]:
tags = list(set(data["Tag"].values))
n_tags = len(tags); n_tags

17

So we have 47959 sentences containing 35178 different words with 17 different tags. We use the SentenceGetter class from last post to retrieve sentences with their labels.

In [7]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [8]:
getter = SentenceGetter(data)

In [9]:
sent = getter.get_next()

This is how a sentence looks now.

In [10]:
print(sent)

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


Okay, that looks like expected, now get all sentences.

In [11]:
sentences = getter.sentences

## 1. Prepare the data

Now we introduce dictionaries of words and tags.

In [12]:
max_len = 75
word2idx = {w: i + 1 for i, w in enumerate(words)}
idx2word = {i + 1: w for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [13]:
word2idx["Obama"]

22849

In [14]:
word2idx["ENDPAD"]

35179

In [15]:
tag2idx["B-geo"]

15

Now we map the senctences to a sequence of numbers and then pad the sequence. Note that we increased the index of the words by one to use zero as a padding value. This is done because we want to use the mask_zeor parameter of the embedding layer to ignore inputs with value zero.

In [16]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]

Using TensorFlow backend.


In [17]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words)

In [18]:
X[0]

array([19622, 34454,  8413,  5032, 22283, 13662, 10666, 12397,   528,
        2791,  1320, 34811, 17334, 23628,  3832,  2791, 12765, 34454,
       19398, 22384,  3263, 24459, 31293, 33853, 35179, 35179, 35179,
       35179, 35179, 35179, 35179, 35179, 35179, 35179, 35179, 35179,
       35179, 35179, 35179, 35179, 35179, 35179, 35179, 35179, 35179,
       35179, 35179, 35179, 35179, 35179, 35179, 35179, 35179, 35179,
       35179, 35179, 35179, 35179, 35179, 35179, 35179, 35179, 35179,
       35179, 35179, 35179, 35179, 35179, 35179, 35179, 35179, 35179,
       35179, 35179, 35179], dtype=int32)

And we need to do the same for our tag sequence.

In [19]:
y = [[tag2idx[w[2]] for w in s] for s in sentences]

In [20]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

For training the network we also need to change the labels y to categorial.

In [21]:
from keras.utils import to_categorical

In [22]:
y = [to_categorical(i, num_classes=n_tags) for i in y]

We split in train and test set.

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)

## 2. Setup the CRF-LSTM

Now we can fit a LSTM-CRF network with an embedding layer.

In [25]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

In [26]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=20, input_length=max_len, mask_zero=True)(input)  # 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True, recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [27]:
model = Model(input, out)

In [28]:
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

/Users/ennoh/anaconda3/envs/py36/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/Users/ennoh/anaconda3/envs/py36/lib/python3.6/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 75)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 75, 20)            703600    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 75, 100)           28400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 75, 50)            5050      
_________________________________________________________________
crf_1 (CRF)                  (None, 75, 17)            1190      
Total params: 738,240
Trainable params: 738,240
Non-trainable params: 0
_________________________________________________________________


In [30]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=5, validation_split=0.1, verbose=1)

Instructions for updating:
Use tf.cast instead.
Train on 38846 samples, validate on 4317 samples
Epoch 1/5
38846/38846 [==============================] - 144s 4ms/step - loss: 0.1076 - crf_viterbi_accuracy: 0.9697 - val_loss: 0.0431 - val_crf_viterbi_accuracy: 0.9848
Epoch 2/5
38846/38846 [==============================] - 142s 4ms/step - loss: 0.0321 - crf_viterbi_accuracy: 0.9882 - val_loss: 0.0275 - val_crf_viterbi_accuracy: 0.9893
Epoch 3/5
38846/38846 [==============================] - 147s 4ms/step - loss: 0.0238 - crf_viterbi_accuracy: 0.9905 - val_loss: 0.0239 - val_crf_viterbi_accuracy: 0.9901
Epoch 4/5
38846/38846 [==============================] - 148s 4ms/step - loss: 0.0208 - crf_viterbi_accuracy: 0.9914 - val_loss: 0.0230 - val_crf_viterbi_accuracy: 0.9903
Epoch 5/5
38846/38846 [==============================] - 149s 4ms/step - loss: 0.0192 - crf_viterbi_accuracy: 0.9919 - val_loss: 0.0220 - val_crf_viterbi_accuracy: 0.9905


In [31]:
hist = pd.DataFrame(history.history)

In [32]:
hist

,val_loss,val_crf_viterbi_accuracy,loss,crf_viterbi_accuracy
0,0.043120,0.984844,0.107617,0.969715
1,0.027528,0.989267,0.032091,0.988193
2,0.023941,0.990147,0.023828,0.990477
3,0.022956,0.990324,0.020803,0.991381
4,0.022035,0.990527,0.019184,0.991882


In [33]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.figure(figsize=(12,12))
plt.plot(hist["crf_viterbi_accuracy"])
plt.plot(hist["val_crf_viterbi_accuracy"])
plt.show()

<Figure size 1200x1200 with 1 Axes>

## 3. Evaluation

Now we can evaluate our model systematically. You can find the details in this post, here we just apply it.

In [34]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

In [35]:
test_pred = model.predict(X_te, verbose=1)

4796/4796 [==============================] - 5s 1ms/step


In [36]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [37]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))

F1-score: 81.8%


In [38]:
print(classification_report(test_labels, pred_labels))

           precision    recall  f1-score   support

      per       0.78      0.76      0.77      1719
      geo       0.85      0.87      0.86      3968
      tim       0.88      0.84      0.86      2013
      gpe       0.96      0.91      0.93      1561
      org       0.68      0.68      0.68      2145
      art       0.00      0.00      0.00        39
      eve       1.00      0.03      0.05        38
      nat       0.00      0.00      0.00        21

micro avg       0.83      0.81      0.82     11504
macro avg       0.82      0.81      0.81     11504



Finally, we look at some predictions.

In [40]:
i = 1927
p = model.predict(np.array([X_te[i]]))
p = np.argmax(p, axis=-1)
true = np.argmax(y_te[i], -1)
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(X_te[i], true, p[0]):
    if w != 0:
        print("{:15}: {:5} {}".format(words[w-1], tags[t], tags[pred]))

Word           ||True ||Pred
Patriarch      : B-org B-per
Bartholomew    : I-org I-per
called         : O     O
it             : O     O
proof          : O     O
that           : O     O
there          : O     O
are            : O     O
no             : O     O
insurmountable : O     O
problems       : O     O
in             : O     O
the            : O     O
Church         : B-tim B-org
of             : I-tim I-org
Christ         : I-tim I-org
.              : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
ENDPAD         : O     O
E

This looks pretty good and it did require any feature engineering. The power of the CRF is not really visible here, but if we had a dataset with more complicated named entites this would be quite strong.

## 4. Inference With The LSTM-CRF

UPDATE: Since many people asked how to use this with new data, which is obviously important, I show you now how to do it. To stay in the frame of this tutorial, I will assume the text is already tokenized. If you don’t know how to do this, I recommend looking at my post about word vectors.

In [41]:
test_sentence = ["Hawking", "was", "a", "Fellow", "of", "the", "Royal", "Society", ",", "a", "lifetime", "member",
                 "of", "the", "Pontifical", "Academy", "of", "Sciences", ",", "and", "a", "recipient", "of",
                 "the", "Presidential", "Medal", "of", "Freedom", ",", "the", "highest", "civilian", "award",
                 "in", "the", "United", "States", "."]

Now we transform every word to it’s integer index. Note that we mapping unknown words to zero. Normally you would want to add a UNKNOWN token to your vocabulary. Then you cut the vocabulary on which you train the model and replace all uncommon words by the UNKNOWN token. We haven’t done this for simplicity.

In [42]:
x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in test_sentence]],
                            padding="post", value=0, maxlen=max_len)

And now we can predict with the model and see what we got.

In [43]:
p = model.predict(np.array([x_test_sent[0]]))
p = np.argmax(p, axis=-1)
print("{:15}||{}".format("Word", "Prediction"))
print(30 * "=")
for w, pred in zip(test_sentence, p[0]):
    print("{:15}: {:5}".format(w, tags[pred]))

Word           ||Prediction
Hawking        : I-gpe
was            : O    
a              : O    
Fellow         : O    
of             : O    
the            : O    
Royal          : B-org
Society        : I-org
,              : O    
a              : O    
lifetime       : O    
member         : O    
of             : O    
the            : O    
Pontifical     : B-org
Academy        : I-org
of             : I-org
Sciences       : I-org
,              : O    
and            : O    
a              : O    
recipient      : O    
of             : O    
the            : O    
Presidential   : O    
Medal          : I-gpe
of             : O    
Freedom        : B-org
,              : O    
the            : O    
highest        : O    
civilian       : O    
award          : O    
in             : O    
the            : O    
United         : B-geo
States         : I-geo
.              : O    


I hope you enjoied this post and learned something that you can apply in your daily work. Next time I show you how you can improve the model even further by using character-level embeddings.